# Classical Perlin Noise

In the classical approach, Perlin noise is generated using deterministic algorithms. Specifically, the simplex function utilizes the OpenSimplex library to create a grid of noise values based on specified parameters. Each point (x,y) in this grid receives a noise value that ranges between 0 and 1. This noise function is known for producing smooth, coherent noise patterns that are used extensively in graphics and procedural generation.

The generated heightmap, Z, is then converted into an image using the height2image function. This function maps the noise values to colors based on predefined terrain thresholds such as sea, beach, grass, forest, mountain, and snow. The resulting image provides a visual representation of the heightmap, making it suitable for further analysis or rendering.

In [ ]:
from PIL import Image
from IPython.display import display

def height2image(Z, terrain=None):
    # Converts a heightmap Z into a PIL image
    # If terrain is None, the image will be black and white
    # Otherwise, terrain thresholds are used to differentiate between various terrain types
    
    image = {}  # Dictionary to store pixel color for each position
    
    for pos in Z:  # Iterate over each position in the heightmap
        if terrain:  # If terrain thresholds are provided
            if Z[pos] < terrain[0]:
                image[pos] = (50,120,200)  # Sea (blue)
            elif Z[pos] < terrain[1]:
                image[pos] = (220,220,10)  # Beach (yellow)
            elif Z[pos] < terrain[2]:
                image[pos] = (100,200,0)   # Grass (green)
            elif Z[pos] < terrain[3]:
                image[pos] = (75,150,0)    # Forest (darker green)
            elif Z[pos] < terrain[4]:
                image[pos] = (200,200,200)  # Mountain (gray)
            else:
                image[pos] = (255,255,255)  # Snow (white)
        else:  # If no terrain thresholds, generate a grayscale image
            z = int(255 * Z[pos])
            image[pos] = (z, z, z)  # Grayscale based on height
    
    # Determine the dimensions of the image from the heightmap
    X = max(Z.keys())[0] + 1
    Y = max(Z.keys())[1] + 1
    img = Image.new('RGB', (X, Y))  # Create a new RGB image with these dimensions
    
    for x in range(img.size[0]):  # Set each pixel in the image
        for y in range(img.size[1]):
            img.load()[x, y] = image[x, y]
    
    return img

def plot_height(Z, terrain=[5/16, 6/16, 9/16, 12/16, 14/16], zoom=None):
    # Display a heightmap as an image with terrain colors by default
    img = height2image(Z, terrain=terrain)  # Generate the image
    
    if zoom:  # If zoom is specified, resize the image accordingly
        img = img.resize((zoom * img.size[0], zoom * img.size[0]), Image.LANCZOS)
    
    img.save('temp.png')  # Save the image temporarily
    display(Image.open('temp.png'))  # Display the image in the notebook


In [ ]:
from opensimplex import OpenSimplex
import random

def simplex(L, period):
    """
    Generate a heightmap using Simplex noise for a grid of size L[0]xL[1].
    
    Parameters:
    - L: Tuple (L[0], L[1]) specifying the dimensions of the heightmap.
    - period: Tuple (period[0], period[1]) defining the period of the noise in x and y directions.
    
    Returns:
    - Z: Dictionary with (x, y) tuples as keys and noise values as the associated values.
    """
    
    # Initialize the Simplex noise generator with a random seed
    gen = OpenSimplex(seed=random.randint(0, 10**20))
    
    # Initialize an empty dictionary to store the heightmap values
    Z = {}
    
    # Loop through each coordinate in the grid
    for x in range(L[0]):
        for y in range(L[1]):
            # Normalize x and y coordinates to range [-0.5, 0.5]
            xx = period[0] * (x / L[0] - 0.5)
            yy = period[1] * (y / L[1] - 0.5)
            
            # Generate Simplex noise value and normalize to range [0, 1]
            Z[x, y] = gen.noise2(xx, yy) / 2 + 0.5
    
    return Z


In [ ]:
# Generate a heightmap with Simplex noise for a grid of size 500x500
Zlow = simplex([500, 500], [10, 10])

# Plot the heightmap
plot_height(Zlow)


In [ ]:
# Generate two heightmaps with Simplex noise
Zlow = simplex([500, 500], [10, 10])
Zhigh = simplex([500, 500], [100, 100])

# Combine the two heightmaps to create a new heightmap (Z)
# Zlow contributes 80% and Zhigh contributes 20% to the final heightmap
Z = {pos: 0.8 * Zlow.get(pos, 0) + 0.2 * Zhigh.get(pos, 0) for pos in Zhigh}

# Plot the combined heightmap (Z) using the plot_height function
plot_height(Z)
